In [1]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
# sys.path.append("..")
sys.path.append("/home/yifan/projects/cophi/ContraVis")
import numpy as np

sys.path.append("/home/yifan/projects/deepdebugertool/DLVisDebugger")
# REF_PATH = "/home/yifan/Exp/Noise/0.2/experiment_vis"
# CONFIG_PATH = "/home/yifan/experiments/noise/20"
# TAR_PATH = "/home/yifan/dataset/cleanfornoise10"

# # CLEAN_PATH = "/home/yifan/Exp/Noise/0.2/experiment1"
TAR_PATH = "/home/yifan/dataset/resnet18_with_dropout/pairflip/cifar10/0"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"



ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis2'

########## initulize reference data and target data
from singleVis.DataInit import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
DEVICE = "cuda:1"
tar_datainit = DataInit(TAR_PATH,TAR_PATH,TAR_EPOCH,DEVICE)
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH,DEVICE)

ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()




/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 432.61it/s]


NET resnet18_with_dropout
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 6298.09it/s]


In [2]:
from scipy.linalg import svd  
def orthogonal_mapping(data1, data2):
        """
        use Orthogonal Mapping, map data2 to data1's space。
    
        :param data1: numpy array, shape (n_samples, n_features)
        :param data2: numpy array, shape (n_samples, n_features)
        :return: data2_mapped: numpy array, mapped data2
        """
        # step1: Centralized data
        data1_centered = data1 - np.mean(data1, axis=0)
        data2_centered = data2 - np.mean(data2, axis=0)
        # step2: Calculate the cross-covariance matrix
        C = data2_centered.T @ data1_centered
        # step3: singular value decomposition
        U, _, Vt = svd(C)

        # step4: Compute orthogonal mapping matrix
        W = U @ Vt

        # step5: Apply mapping matrix
        data2_mapped = data2_centered @ W
    
        return data2_mapped
X_train1 = ref_provider.train_representation(REF_EPOCH)
X_train2 = tar_provider.train_representation(TAR_EPOCH)
X_train2 = X_train2.reshape(X_train2.shape[0],X_train2.shape[1])
print(X_train1.shape)
print(X_train2.shape)
data2_mapped = orthogonal_mapping(X_train1,X_train2)

(50000, 512)
(50000, 512)


In [7]:
from eval.evaluate import *
rate = evaluate_high_dimesion_trans_knn_preserving(data2_mapped, X_train2)
print(rate)

0.9999986666666667


In [10]:
from contrast.transfomration import *
tarns_model,tar_mapped,ref_reconstructed  = transformation_train(X_train1,X_train2)

Epoch [1/200], Loss: 1.4547
Epoch [2/200], Loss: 1.3728
Epoch [3/200], Loss: 1.2530
Epoch [4/200], Loss: 1.1483
Epoch [5/200], Loss: 1.1644
Epoch [6/200], Loss: 1.0922
Epoch [7/200], Loss: 1.0067
Epoch [8/200], Loss: 0.9752
Epoch [9/200], Loss: 0.9610
Epoch [10/200], Loss: 0.9349
Epoch [11/200], Loss: 0.9039
Epoch [12/200], Loss: 0.8864
Epoch [13/200], Loss: 0.8732
Epoch [14/200], Loss: 0.8514
Epoch [15/200], Loss: 0.8357
Epoch [16/200], Loss: 0.8287
Epoch [17/200], Loss: 0.8180
Epoch [18/200], Loss: 0.8037
Epoch [19/200], Loss: 0.7955
Epoch [20/200], Loss: 0.7897
Epoch [21/200], Loss: 0.7789
Epoch [22/200], Loss: 0.7694
Epoch [23/200], Loss: 0.7631
Epoch [24/200], Loss: 0.7549
Epoch [25/200], Loss: 0.7468
Epoch [26/200], Loss: 0.7401
Epoch [27/200], Loss: 0.7308
Epoch [28/200], Loss: 0.7211
Epoch [29/200], Loss: 0.7135
Epoch [30/200], Loss: 0.7052
Epoch [31/200], Loss: 0.6967
Epoch [32/200], Loss: 0.6888
Epoch [33/200], Loss: 0.6799
Epoch [34/200], Loss: 0.6708
Epoch [35/200], Loss: 0

In [4]:
ref_pred = ref_provider.get_pred(REF_EPOCH, X_train1).argmax(axis=1)
tar_pred = tar_provider.get_pred(TAR_EPOCH, X_train2).argmax(axis=1)

100%|██████████| 250/250 [00:00<00:00, 6600.38it/s]


In [5]:
ref_reconstructed

array([[0.5552391 , 1.3357137 , 0.        , ..., 1.4515507 , 1.2634635 ,
        0.        ],
       [1.4458725 , 0.30887666, 2.0563803 , ..., 0.        , 0.97767997,
        0.        ],
       [1.4317726 , 0.17496452, 1.501822  , ..., 0.        , 1.0427316 ,
        0.        ],
       ...,
       [1.1769273 , 0.        , 1.1185749 , ..., 0.        , 0.        ,
        0.        ],
       [1.1797391 , 0.13735478, 1.4692004 , ..., 0.20582874, 0.6571957 ,
        2.3002887 ],
       [0.84440976, 0.2670025 , 1.9620161 , ..., 0.        , 1.078705  ,
        1.0086372 ]], dtype=float32)

In [7]:
from eval.evaluate import *
rate = evaluate_high_dimesion_trans_knn_preserving(tar_mapped, X_train2)
print(rate)

0.500748


In [12]:

reconstructed_pred = tar_provider.get_pred(TAR_EPOCH, ref_reconstructed).argmax(axis=1)

100%|██████████| 250/250 [00:00<00:00, 6842.17it/s]


In [13]:
diff = 0
for i in range(len(reconstructed_pred)):
    if reconstructed_pred[i] != tar_pred[i]:
        diff = diff+1
print(diff)

3


In [14]:
from sklearn.neighbors import NearestNeighbors

sub_len = 50000

# 1. use cos sim build 
model = NearestNeighbors(n_neighbors=1, metric='cosine', algorithm='auto').fit(tar_mapped[:sub_len])

# 2. find X_train1 most sim point data2_mapped中的最相似点
distances, most_similar_points = model.kneighbors(X_train1[:sub_len])

most_similar_points = most_similar_points.flatten()  # 将索引数组转为1D

# 3. 比较预测值
m = np.sum(ref_pred[:sub_len] != tar_pred[most_similar_points])
k = np.sum(ref_pred[:sub_len] != tar_pred[:sub_len])

print("diff pred number is {}".format(m))
print("index diff pred number is {}".format(k))


diff pred number is 41
index diff pred number is 12
